In [ ]:
# Install a pip package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/ngargaud/forge-api.git
!{sys.executable} -m pip install pillow # to show images in the notebook

In [ ]:
import IPython
import json
import time
import asyncvnc

from ai_api import AiApi

ollama_image_path = "/tmp/screenshot.png"
which_image_prompt = """
Fill the following json describing the image:
{
    "isDesktop": set to true if there is no windows openned,
    "isBrowserOpen": set to true if the browser is open and visible,
    "description": string
}
"""

check_description_prompt = """
Fill the following json with the following informations:
{
    "isDesktop": set to true if there is no windows openned,
    "isBrowserOpen": set to true if the browser is open and visible,
}

"""

api = AiApi()
ollama = api.get_client("ollama")
print("Pulling llava")
ollama.pull('llava:34b-v1.6')
print("Pulling mistral")
ollama.pull('mistral:instruct')
url, port = api.get_api_url("sandbox").rsplit(":", 1)
async with asyncvnc.connect(url, int(port)) as client:
    client.mouse.move(0,0)

    chromeIcon = {"x":80, "y":520}
    closeButton = {"x":235, "y":60}
    # get image from the sandbox and save it for ollama vision model
    screenshot = await api.get_sandbox_screenshot(filename=ollama_image_path, client=client)
    IPython.display.display(screenshot)

    # get information about the current status of the sandbox with a vision model
    stream = ollama.generate(model='llava:v1.6', prompt=which_image_prompt, images=[ollama_image_path], stream=True, format="json")
    status = ""
    for chunk in stream:
        status += chunk['response']
        # print(chunk['response'], end='', flush=True)
    try:
        status_js = json.loads(status)
        print("")
        # ask an other model
        checking = ollama.generate(model='llava:v1.6', prompt=check_description_prompt + status_js.get("description"), images=[ollama_image_path], stream=False, format="json")
        # checking = ollama.generate(model='mistral:instruct', prompt=check_description_prompt + status_js.get("description"), stream=False, format="json")
        # print(checking.get("response"))
        checking_js = json.loads(checking.get("response"))
    
    except Exception as e:
        print("")
        print("ERROR: can not parse the status", e)
        print(status)
        print("-"*10)
        print(checking)
        status_js = None
    # if we are able to retreive the status then do if/else
    if status_js and checking_js:
        if status_js.get("isBrowserOpen") and checking_js.get("isBrowserOpen") or checking_js.get("isBrowserOpen"):
            print("Decision: Close the browser", status, checking.get("response"))
            client.mouse.move(closeButton.get("x", 0), closeButton.get("y", 0))
            client.mouse.click()
        elif status_js.get("isBrowserOpen") and checking_js.get("isBrowserOpen"):
            print("Decision: not sure about what to do, model disagree", status, checking.get("response"))
        else:
            print("Decision: need to open the browser", status, checking.get("response"))          
            #open the chrome browser
            client.mouse.move(chromeIcon.get("x", 0), chromeIcon.get("y", 0))
            client.mouse.click()
            client.mouse.click()
        time.sleep(1)
    
        # get image from the sandbox and save it for ollama vision model
        screenshot = await api.get_sandbox_screenshot(filename=ollama_image_path, client=client)
        IPython.display.display(screenshot)
